![delensalot logo](./../../res/dlensalot2.png)

# Interactive minimal working example - quickstart
In this notebook, you generate lensed CMB simulations on the full sky inclusive of isotropic white noise and without foregrounds.
At the end of this tutorial, you will plot the power spectra of the CMB and lensing potential realizations.


# `map2tempblm()`
 * generate obs and sky maps
 * map2tempblm()
 * check residual lensing amplitude

In [3]:
from IPython.display import clear_output
from datetime import datetime
import os
from os.path import join as opj

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

import delensalot
from delensalot import utils
from delensalot.run import run
from delensalot.config.metamodel.dlensalot_mm import DLENSALOT_Model, DLENSALOT_Analysis, DLENSALOT_Itrec

delensalot_model = DLENSALOT_Model(defaults_to='P_FS_CMBS4', analysis = DLENSALOT_Analysis(key='p_p', TEMP_suffix='my_first_delensalot_analysis_quickstart'), itrec = DLENSALOT_Itrec(itmax=3))
delensalot_runner = run(config_fn='', job_id='generate_sim', config_model=delensalot_model, verbose=True)
delensalot_runner.run()
ana_mwe = delensalot_runner.init_job()
bsky = ana_mwe.simulationdata.get_sim_sky(simidx=0, field='polarization', space='alm', spin=0)[1]
obs = ana_mwe.simulationdata.get_sim_obs(simidx=0, field='polarization', space='map', spin=2)

clear_output(wait=True)
print("Cell finished {}".format(datetime.now().strftime("%H:%M:%S")))

Cell finished 11:51:45


In [ ]:
blt = delensalot.map2tempblm(obs, lmax_cmb=delensalot_model.analysis.lm_max_ivf[0], beam=delensalot_model.analysis.beam, itmax=delensalot_model.itrec.itmax, noise={'P':delensalot_model.noisemodel.nlev_p}, use_approximateWF=True, defaults_to='P_FS_TEST', verbose=True, )

clear_output(wait=True)
print("Cell finished {}".format(datetime.now().strftime("%H:%M:%S")))

In [5]:
input = hp.anafast(bsky, lmax=200)
output = hp.anafast(bsky-hp.alm2map(blt, nside=2048), lmax=200)

Al = np.mean(output[30:200]/input[30:200])
print(Al)

0.13161561089786333


In [3]:
import delensalot.config.metamodel.dlensalot_mm as dm

In [4]:
dm.__dict__

{'__name__': 'delensalot.config.metamodel.dlensalot_mm',
 '__doc__': 'dlensalot_mm.py: Contains classes defining the metamodel of the Dlensalot formalism.\n    The metamodel is a structured representation, with the `DLENSALOT_Model` as the main building block.\n    We use the attr package. It provides handy ways of validation and defaulting.\n',
 '__package__': 'delensalot.config.metamodel',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7f56e97f05d0>,
 '__spec__': ModuleSpec(name='delensalot.config.metamodel.dlensalot_mm', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f56e97f05d0>, origin='/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/delensalot/delensalot/config/metamodel/dlensalot_mm.py'),
 '__file__': '/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/delensalot/delensalot/config/metamodel/dlensalot_mm.py',
 '__cached__': '/mnt/c/Users/sebas/OneDrive/Desktop/Uni/ext/git/delensalot/delensalot/config/metamodel/__pycache__/dlensalot_mm.cpython-311.

# `anafast()`
 * generate obs and sky maps
 * anafast()
 * check residual lensing amplitude

In [ ]:
from IPython.display import clear_output
from datetime import datetime
import os
from os.path import join as opj

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

import delensalot
from delensalot.run import run
from delensalot.config.metamodel.dlensalot_mm import DLENSALOT_Model, DLENSALOT_Analysis, DLENSALOT_Itrec

delensalot_model = DLENSALOT_Model(defaults_to='P_FS_CMBS4', analysis = DLENSALOT_Analysis(key='p_p', TEMP_suffix='my_first_delensalot_analysis_quickstart'), itrec = DLENSALOT_Itrec(itmax=3))
delensalot_runner = run(config_fn='', job_id='generate_sim', config_model=delensalot_model, verbose=True)
delensalot_runner.run()
ana_mwe = delensalot_runner.init_job()
bsky = ana_mwe.simulationdata.get_sim_sky(simidx=0, field='polarization', space='map', spin=0)[1]
obs = ana_mwe.simulationdata.get_sim_obs(simidx=0, field='polarization', space='map', spin=2)

noise = {'P':delensalot_model.noisemodel.nlev_p}
lmax_cmb = delensalot_model.analysis.lm_max_ivf[0]
beam = delensalot_model.analysis.beam
itmax = delensalot_model.itrec.itmax

In [ ]:
import delensalot
delblm = delensalot.anafast(
    obs,
    lmax_cmb=lmax_cmb,
    beam=beam,
    itmax=itmax,
    noise=noise,
    use_approximateWF=True,
    defaults_to='P_FS_TEST',
    verbose=True, )

In [ ]:
input = hp.anafast(bsky, lmax=200)
output = hp.anafast(bsky-hp.alm2map(blt, nside=1024), lmax=200)

Al = np.mean(output[30:200]/input[30:200])
print(Al, self.Al_assert[job_id][key])

This concludes the tutorial.

# Summary and next steps
This tutorial demonstrated how to generate delensed maps in a quick way and without knowing much about delensalot.

Check out the other tutorials, or see the [documentation](link_to_rtd_doc), or [github](https://github.com/NextGenCMB/delensalot).